In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

In [2]:
!pip install torch

In [3]:
!pip install protobuf==3.20.0

In [13]:
data = pd.read_csv('09-03-23_to_15-03-23.csv')
data = data[['text', 'datetime']]

In [14]:
def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [15]:

data['process_text'] = data.text
# data['negative_bert'] = data.text
# data['neutral_bert'] = data.text
# data['positive_bert'] = data.text

#use the preprocess to clean the data
data['process_text'] = data['text'].apply(preprocess)

In [16]:
data

,text,datetime,process_text
0,Orange coin rising.\n\nEnergy will soon be pri...,2020-09-01 00:09:57+00:00,Orange coin rising. Energy will soon be price...
1,#BITCOIN #FLASHES #SIGNS OF #TECHNICAL #STRENG...,2020-09-01 00:09:54+00:00,#BITCOIN #FLASHES #SIGNS OF #TECHNICAL #STRENG...
2,A new #Bitcoin price zone has just been detect...,2020-09-01 00:09:49+00:00,A new #Bitcoin price zone has just been detect...
3,🤝 Follow me on @betfury_io. Let's hunt for Bit...,2020-09-01 00:09:18+00:00,🤝 Follow me on Let's hunt for Bitcoins togeth...
4,"Closed Buy EURUSD 1.19422 for +1.4 pips, total...",2020-09-01 00:08:08+00:00,"Closed Buy EURUSD 1.19422 for +1.4 pips, total..."
...,...,...,...
77827,@hodlberry No wallet. Just store the passphras...,2020-12-31 16:00:02+00:00,No wallet. Just store the passphrase+seed in ...
77828,Deposit Cryptocurrency on a Betting Site #bitc...,2020-12-31 16:00:01+00:00,Deposit Cryptocurrency on a Betting Site #bitc...
77829,"The price of\nSpaghetti alla Carbonara\nis 38,...",2020-12-31 16:00:01+00:00,"The price of Spaghetti alla Carbonara is 38,16..."
77830,🎆 #HappyNewYear 2021 From Huobi!\n\nThank you ...,2020-12-31 16:00:01+00:00,🎆 #HappyNewYear 2021 From Huobi! Thank you fo...


In [17]:
!pyenv local crypto

In [18]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [19]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [22]:
data = data[:1000]
data['text'] = data['process_text'].apply(scores_bert)

/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_7143/1353871001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['process_text'].apply(scores_bert)


In [24]:
data = data[['datetime','process_text','text']]
data

,datetime,process_text,text
0,2020-09-01 00:09:57+00:00,Orange coin rising. Energy will soon be price...,"[0.0011185623, 0.76268345, 0.23619792]"
1,2020-09-01 00:09:54+00:00,#BITCOIN #FLASHES #SIGNS OF #TECHNICAL #STRENG...,"[0.0004626749, 0.71313405, 0.28640324]"
2,2020-09-01 00:09:49+00:00,A new #Bitcoin price zone has just been detect...,"[0.00021627804, 0.92800385, 0.071779884]"
3,2020-09-01 00:09:18+00:00,🤝 Follow me on Let's hunt for Bitcoins togeth...,"[0.0007858043, 0.56673324, 0.43248105]"
4,2020-09-01 00:08:08+00:00,"Closed Buy EURUSD 1.19422 for +1.4 pips, total...","[0.0009984833, 0.6328706, 0.36613083]"
...,...,...,...
995,2020-09-01 16:00:59+00:00,"Join our faucet, earn with tasking Follow, RT,...","[0.00071918394, 0.6683136, 0.33096728]"
996,2020-09-01 16:00:52+00:00,#BITCOIN FOR THE #BITCOIN GOD! SATS FOR THE SA...,"[0.04297031, 0.5217078, 0.43532184]"
997,2020-09-01 16:00:51+00:00,Please I need info from anyone that has regist...,"[0.0002819477, 0.6348894, 0.36482865]"
998,2020-09-01 16:00:42+00:00,Ready for $15k #bitcoin?,"[0.12720288, 0.68602765, 0.18676946]"


In [36]:
#modifying the datetime
def dateonly(x):
    return x[:10]
data['date'] = data['datetime'].apply(dateonly)

data

/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_7143/1637026699.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['datetime'].apply(dateonly)


,datetime,process_text,text,date
0,2020-09-01 00:09:57+00:00,Orange coin rising. Energy will soon be price...,"[0.0011185623, 0.76268345, 0.23619792]",2020-09-01
1,2020-09-01 00:09:54+00:00,#BITCOIN #FLASHES #SIGNS OF #TECHNICAL #STRENG...,"[0.0004626749, 0.71313405, 0.28640324]",2020-09-01
2,2020-09-01 00:09:49+00:00,A new #Bitcoin price zone has just been detect...,"[0.00021627804, 0.92800385, 0.071779884]",2020-09-01
3,2020-09-01 00:09:18+00:00,🤝 Follow me on Let's hunt for Bitcoins togeth...,"[0.0007858043, 0.56673324, 0.43248105]",2020-09-01
4,2020-09-01 00:08:08+00:00,"Closed Buy EURUSD 1.19422 for +1.4 pips, total...","[0.0009984833, 0.6328706, 0.36613083]",2020-09-01
...,...,...,...,...
995,2020-09-01 16:00:59+00:00,"Join our faucet, earn with tasking Follow, RT,...","[0.00071918394, 0.6683136, 0.33096728]",2020-09-01
996,2020-09-01 16:00:52+00:00,#BITCOIN FOR THE #BITCOIN GOD! SATS FOR THE SA...,"[0.04297031, 0.5217078, 0.43532184]",2020-09-01
997,2020-09-01 16:00:51+00:00,Please I need info from anyone that has regist...,"[0.0002819477, 0.6348894, 0.36482865]",2020-09-01
998,2020-09-01 16:00:42+00:00,Ready for $15k #bitcoin?,"[0.12720288, 0.68602765, 0.18676946]",2020-09-01


In [37]:
data["negative_bert"] = data["text"].map(lambda x: x[0])
data["neutral_bert"] = data["text"].map(lambda x: x[1])
data["positive_bert"] = data["text"].map(lambda x: x[2])
data = data[['date', 'process_text', 'negative_bert', 'neutral_bert','positive_bert']]
data

/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_7143/3230259926.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["negative_bert"] = data["text"].map(lambda x: x[0])
/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_7143/3230259926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["neutral_bert"] = data["text"].map(lambda x: x[1])
/var/folders/dn/rwn91pwx3q9c4zbfkf6rfb6h0000gn/T/ipykernel_7143/3230259926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

,date,process_text,negative_bert,neutral_bert,positive_bert
0,2020-09-01,Orange coin rising. Energy will soon be price...,0.001119,0.762683,0.236198
1,2020-09-01,#BITCOIN #FLASHES #SIGNS OF #TECHNICAL #STRENG...,0.000463,0.713134,0.286403
2,2020-09-01,A new #Bitcoin price zone has just been detect...,0.000216,0.928004,0.071780
3,2020-09-01,🤝 Follow me on Let's hunt for Bitcoins togeth...,0.000786,0.566733,0.432481
4,2020-09-01,"Closed Buy EURUSD 1.19422 for +1.4 pips, total...",0.000998,0.632871,0.366131
...,...,...,...,...,...
995,2020-09-01,"Join our faucet, earn with tasking Follow, RT,...",0.000719,0.668314,0.330967
996,2020-09-01,#BITCOIN FOR THE #BITCOIN GOD! SATS FOR THE SA...,0.042970,0.521708,0.435322
997,2020-09-01,Please I need info from anyone that has regist...,0.000282,0.634889,0.364829
998,2020-09-01,Ready for $15k #bitcoin?,0.127203,0.686028,0.186769


In [38]:
grouped_data = pd.DataFrame(data.groupby(['date'])[['negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

In [39]:
grouped_data

,date,negative_bert,neutral_bert,positive_bert
0,2020-09-01,56.146008,350.283112,237.570877
1,2020-09-02,17.743044,170.996780,106.260170
2,2020-09-03,3.225846,37.338848,19.435305
3,datetime,0.145655,0.424722,0.429624
